In [11]:
from pytest import importorskip
from train_collections import DS_HARAKAT , DS_ARABIC_LETTERS
from text_encoder import TextEncoder
import pandas as pd

In [3]:

    
output_encoder = TextEncoder(DS_HARAKAT)
input_encoder = TextEncoder(DS_ARABIC_LETTERS)

c = input_encoder.encode("السلام عليكم")
input_encoder.decode(c)



'ا ل س ل ا م   ع ل ي ك م'

In [18]:
from math import e
import numpy as np


input_file = 'clean_out/X.csv'
output_file = 'clean_out/Y.csv'

# input is a sequence of letters and output is a sequence of harakat

def read_data(input_file, output_file):
    X = []
    Y = []
    # read csv files 
    with open(input_file, 'r') as f:
        X = f.readlines()
    with open(output_file, 'r') as f:
        Y = f.readlines()
        
    # remove the \n from the end of each line
    X = [x.strip().split(',') for x in X]
    Y = [y.strip().split(',') for y in Y]
    
    return X, Y


x,y = read_data(input_file, output_file)
print(x[0])
encoded_x = input_encoder.encode(x[0])
print(encoded_x)
d = input_encoder.decode(encoded_x)
d

from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, x, y, input_encoder, output_encoder):
        self.x = x
        self.y = y
        self.input_encoder = input_encoder
        self.output_encoder = output_encoder
        self.fixed_len = np.max([len(x) for x in self.x])
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        
        # pad the sequence to fixed length
        x = x + ['P'] * (self.fixed_len - len(x))
        y = y + ['P'] * (self.fixed_len - len(y))
        
        
        x = self.input_encoder.encode(x)
        y = self.output_encoder.encode(y)
        
        return x, y

dataset = MyDataset(x, y, input_encoder, output_encoder)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

for x, y in dataloader:
    print(x)
    print(y)
    break

['ق', 'و', 'ل', 'ه', ' ', 'أ', 'و', ' ', 'ق', 'ط', 'ع', ' ', 'ا', 'ل', 'أ', 'و', 'ل', ' ', 'ي', 'د', 'ه', ' ', 'إ', 'ل', 'خ', ' ', 'ق', 'ا', 'ل', ' ', 'ا', 'ل', 'ز', 'ر', 'ك', 'ش', 'ي']
[20, 26, 22, 25, 36, 31, 26, 36, 20, 15, 17, 36, 0, 22, 31, 26, 22, 36, 27, 7, 25, 36, 32, 22, 6, 36, 20, 0, 22, 36, 0, 22, 10, 9, 21, 12, 27]
[tensor([26, 20, 26, 26, 26, 26, 20, 22, 19, 20]), tensor([25, 26, 23, 22, 27, 20, 26, 25,  9, 26]), tensor([ 8, 22,  3, 26,  4,  0, 22, 36, 17, 22]), tensor([ 0, 25, 22, 36,  0, 22, 25,  0, 36, 25]), tensor([36, 36, 36, 18,  1, 36, 36, 22, 12, 36]), tensor([ 0,  6,  0, 13, 36,  0, 26,  5, 25, 11]), tensor([22, 22, 22,  1,  1,  1, 20, 24,  7, 26]), tensor([23,  0, 23, 36, 31, 24, 19, 36,  2,  0]), tensor([17, 19,  9, 17, 24, 36, 36, 23, 36, 33]), tensor([24,  0,  2, 13, 25, 17, 17,  3, 17, 36]), tensor([28, 36,  7, 27, 36,  1, 23, 22, 22, 21]), tensor([36, 22, 36,  9, 22,  0,  9, 36, 28,  0]), tensor([21,  0,  0,  0, 23, 11, 36, 31, 36, 24]), tensor([ 3,  1, 22, 

In [ ]:
# Training the model 
# we will use a simple RNN model with 2 layers
# the input is a sequence of letters and the output is a sequence of harakat

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

class EncoderRNN(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        #self.out = nn.Linear(hidden_size, output_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        #print(input.shape)
        #print(hidden.shape)
        embedded = self.embedding(input).view(1, 1, -1)
        #print(embedded.shape)
        output = embedded
        output, hidden = self.gru(output, hidden)
        #print(output.shape)
        #print(hidden.shape)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    



